<a href="https://colab.research.google.com/github/Andron00e/Fine-Tuning-project/blob/main/TA_merge_and_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece
!pip install datasets
!pip uninstall wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.4 MB/s eta 0:00:00


In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
HUGGING_FACE_USER_NAME = "Andron00e"

model_name = "Llama-Translation-Answering-adapter"

In [6]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=False, torch_dtype=torch.bfloat16,  device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
qa_model = PeftModel.from_pretrained(model, peft_model_id) # load LoRA adapters

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [7]:
merged_model = qa_model.merge_and_unload()

In [8]:
merged_model_name = "Llama-Translation-Answering-v2"

In [9]:
tokenizer

LlamaTokenizerFast(name_or_path='openlm-research/open_llama_3b', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=False)

In [10]:
torch.cuda.empty_cache()

In [13]:
merged_model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{merged_model_name}", safe_serialization=True, use_auth_token=True)
tokenizer.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{merged_model_name}", use_auth_token=True)

model.safetensors:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Andron00e/Llama-Translation-Answering-v2/commit/66dc81f909934210629005aa6256cd267de7a541', commit_message='Upload tokenizer', commit_description='', oid='66dc81f909934210629005aa6256cd267de7a541', pr_url=None, pr_revision=None, pr_num=None)

Evaluation

In [1]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness

fatal: destination path 'lm-evaluation-harness' already exists and is not an empty directory.


In [2]:
%cd lm-evaluation-harness

/content/lm-evaluation-harness


In [16]:
!pip install -e ".[multilingual]"

Obtaining file:///content/lm-evaluation-harness
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6

In [3]:
!python main.py \
    --model hf-causal \
    --model_args pretrained=Andron00e/Llama-Translation-Answering-v2 \
    --tasks hellaswag \
    --device cuda:0

2023-07-22 13:05:38.380327: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[dynet] random seed: 1234
[dynet] allocating memory: 32MB
[dynet] memory allocation done.
Selected Tasks: ['hellaswag']
Using device 'cuda:0'
Extracting data files: 100% 3/3 [00:00<00:00, 2146.52it/s]
Dataset hellaswag downloaded and prepared to /root/.cache/huggingface/datasets/hellaswag/default/0.1.0/512a66dd8b1b1643ab4a48aa4f150d04c91680da6a4096498a5e5f799623d5ae. Subsequent calls will reuse this data.
100% 3/3 [00:00<00:00, 133.45it/s]
Task: hellaswag; number of docs: 10042
Task: hellaswag; document 0; context prompt (starting on next line):
Personal Care and Style: How to increase breast size with a bra. Check your bra size. Wearing a bra that is too big will not make your breasts look larger. That is why it is important to wear the right size bra for you.
(end of prompt on previous line)
Requests: [Req_loglikelihood('Personal Care and Style: How to increase 